In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchsampler import ImbalancedDatasetSampler
import matplotlib.pyplot as plt
import os
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from functools import reduce
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
import pandas as pd
import sklearn 

In [2]:
import loaders as l
import models as m
import train_eval as te

In [3]:
train_dir = "C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/train_images"
test_dir = "C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/test_images"

transform = transforms.Compose(
    [transforms.Grayscale(), 
     transforms.ToTensor(), 
     transforms.Normalize(mean=(0,),std=(1,))])

valid_size = 0.2
batch_size = 32

train_loader,valid_loader,test_loader=l.make_all_loaders(train_dir,test_dir,transform,valid_size,batch_size)
classes = ('noface','face')

print(train_loader.dataset.targets.count(0))
print(train_loader.dataset.targets.count(1))


26950
64770


In [4]:
net = m.BaseNet_v4()
print(net)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer =optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optim.Adam(net.parameters(), lr=0.001)
#optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

epochs=50

#Training
print("Started Training")
all_losses,all_accuracies,all_f1scores,all_roc_aucc,best_params= te.train_v1(net,criterion,optimizer,epochs,train_loader,valid_loader,device)
            
#plt.plot(all_losses, color='blue')
#plt.plot(all_f1scores, color='red')
plt.subplot(2, 1, 1)
plt.plot(all_f1scores, color='red')
plt.title('Evolution of f1 score during epochs')
plt.subplot(2, 1, 2)
plt.plot(all_losses, color='blue')
plt.title('Evolution of losses during epochs')
plt.tight_layout()


print("Finished Training")

BaseNet_v4(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 20, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(20, 18, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(18, 16, kernel_size=(3, 3), stride=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=400, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)
cuda:0


  0%|          | 0/50 [00:00<?, ?it/s]

Started Training



0it [00:00, ?it/s]
1it [00:00,  1.01it/s]
9it [00:01,  1.44it/s]
17it [00:01,  2.04it/s]
24it [00:01,  2.88it/s]
32it [00:01,  4.04it/s]
40it [00:01,  5.64it/s]
48it [00:01,  7.81it/s]
56it [00:01, 10.69it/s]
64it [00:01, 14.36it/s]
71it [00:01, 18.58it/s]
78it [00:02, 23.34it/s]
86it [00:02, 29.45it/s]
93it [00:02, 35.50it/s]
100it [00:02, 41.40it/s]
108it [00:02, 47.49it/s]
115it [00:02, 49.06it/s]
122it [00:02, 49.50it/s]
129it [00:02, 51.56it/s]
137it [00:02, 57.22it/s]
145it [00:03, 61.45it/s]
152it [00:03, 62.66it/s]
160it [00:03, 65.41it/s]
167it [00:03, 65.69it/s]
175it [00:03, 68.21it/s]
183it [00:03, 69.94it/s]
191it [00:03, 70.65it/s]
199it [00:03, 69.95it/s]
207it [00:03, 71.97it/s]
215it [00:04, 72.28it/s]
223it [00:04, 72.80it/s]
231it [00:04, 72.47it/s]
239it [00:04, 69.60it/s]
247it [00:04, 71.62it/s]
255it [00:04, 72.41it/s]
263it [00:04, 73.78it/s]
271it [00:04, 72.46it/s]
279it [00:04, 72.81it/s]
287it [00:05, 72.08it/s]
295it [00:05, 74.15it/s]
305it [00:05, 57.21i

KeyboardInterrupt: 

In [ ]:
#sauvegarder celui avec les meilleures metrics et pas le dernier

In [ ]:
saved_model_path='C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/test_v4_v2'
torch.save(best_params, saved_model_path )

In [ ]:
net = m.BaseNet_v4()
net.load_state_dict(torch.load(saved_model_path))
net.to(device)
print(net)
#saved_model_path='C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/BaseNet_v2_lo0001_30ep'
#torch.save(net.state_dict(), saved_model_path )

In [ ]:

plt.plot(all_roc_aucc, color='blue')
plt.title('ROC curve during epochs')
#f1,_,_,_,roc = te.calc_metrics_v1(net,test_loader,False)
#print(f1)
#print(roc)
#s=roc_auc_score(all_labels, all_predicted)
#print(s)

In [ ]:
import seaborn as sns
_,_,_,cm,_=te.calc_metrics_v1(net,test_loader,True,device) 
sns.heatmap(cm, annot=True, fmt='.2%', cmap='Blues')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))